# Day 2015_18: Like a GIF For Your Yard 

In [1]:
year = 2015
day  = 18

In [2]:
from local_settings import load_input
content = load_input(year, day)
print(f"[{content[:100]}...]")

Reading [https://adventofcode.com/2015/day/18/input]
 With proxy: [https: http://proxyseso.scania.com:8080]
 With proxy: [http: http://proxyseso.scania.com:8080]
10100 characters read.
[####.#.##.###.#.#.##.#..###.#..#.#.#..##....#.###...##..###.##.#.#.#.##...##..#..#....#.#.##..#...##...]


# Part 1

In [3]:
class Grid:
    NEIGHBOURS = tuple((dx, dy) 
                       for dx in range(-1, 2) 
                       for dy in range(-1, 2) if not (dx == 0 and dy == 0))

    def __init__(self, s):
        self.litLamps = set()
        self.nxtLitLamps = set()
        self.width = 0
        self.height = 0
        self.parse(s)
    
    def parse(self, s):
        lines = list(s.splitlines())
        self.height = len(lines)
        self.width = len(lines[0])
        for y, line in enumerate(lines):
            for x, ch in enumerate(line):
                if ch == '#':
                    self.turnOn(x, y)
        self.swap()

    def turnOn(self,  x, y):
        if self.isOn(x, y):
            return
        self.nxtLitLamps.add((x, y))
            
    def turnOff(self, x, y):
        if self.isOff(x, y):
            return
        self.nxtLitLamps.remove((x, y))
    
    def isOn(self, x, y):
        return (x, y) in self.litLamps
    
    def isOff(self, x, y):
        return (x, y) not in self.litLamps
    
    def noLitsAround(self, x, y):
        return sum((self.isOn(x + dx, y + dy) for dx, dy in self.NEIGHBOURS))
        
    def swap(self):
        self.litLamps = self.nxtLitLamps
        self.nxtLitLamps = set(self.litLamps)
    
    def step(self):
        for x in range(self.width):
            for y in range(self.height):
                if self.isOn(x, y) and self.noLitsAround(x, y) not in (2, 3):
                    self.turnOff(x, y)
                elif self.isOff(x, y) and self.noLitsAround(x, y) == 3:
                    self.turnOn(x, y)
        self.swap()
        
    def countOn(self):
        return len(self.litLamps)
        
    def __str__(self):
        lines = []
        lines2 = []
        for y in range(self.height):
            line = []
            line2 = []
            for x in range(self.width):
                if self.isOn(x, y):
                    line.append("#")
                else:
                    line.append(".")
                line2.append(str(self.noLitsAround(x, y)))
            lines.append(''.join(line))
            lines2.append(''.join(line2))
        lines.append('')
        lines.extend(lines2)
        return '\n'.join(lines)

## Examples:
```
Initial state:
.#.#.#
...##.
#....#
..#...
#.#..#
####..

After 1 step:
..##..
..##.#
...##.
......
#.....
#.##..

After 2 steps:
..###.
......
..###.
......
.#....
.#....

After 3 steps:
...#..
......
...#..
..##..
......
......

After 4 steps:
......
......
..##..
..##..
......
......
```

In [4]:
example = """.#.#.#
...##.
#....#
..#...
#.#..#
####.."""
grid = Grid(example)
print(grid)
for i in range(4):
    print(f'\nStep {i + 1}:')
    grid.step()
    print(grid)
    print(f"Lamps on: {grid.countOn()}")


.#.#.#
...##.
#....#
..#...
#.#..#
####..

103241
223243
022331
241222
264420
243221

Step 1:
..##..
..##.#
...##.
......
#.....
#.##..

023331
024551
013332
111221
132210
131110
Lamps on: 11

Step 2:
..###.
......
..###.
......
.#....
.#....

011211
024642
011211
123321
212000
212000
Lamps on: 8

Step 3:
...#..
......
...#..
..##..
......
......

001010
002220
013220
012220
012210
000000
Lamps on: 4

Step 4:
......
......
..##..
..##..
......
......

000000
012210
023320
023320
012210
000000
Lamps on: 4


In [5]:
grid = Grid(content)
for i in range(100):
    grid.step()
print(f"Lamps on: {grid.countOn()}")


Lamps on: 768


# Part 2

In [6]:
class BrokenGrid(Grid):
    def isCorner(self, x, y):
        return x in (0, self.width - 1) and y in (0, self.height - 1)
    def turnOn(self, x, y):
        if self.isCorner(x, y):
            return
        super().turnOn(x, y)
    def turnOff(self, x, y):
        if self.isCorner(x, y):
            return
        super().turnOff(x, y)
    def isOn(self, x, y):
        if self.isCorner(x, y):
            return True
        return super().isOn(x, y)
    def swap(self):
        super().swap()
        self.litLamps |= {(0, 0), (self.width - 1, 0), 
                          (self.width - 1, self.height - 1), (0, self.height - 1)}

## Examples:
```
Initial state:
##.#.#
...##.
#....#
..#...
#.#..#
####.#

After 1 step:
#.##.#
####.#
...##.
......
#...#.
#.####

After 2 steps:
#..#.#
#....#
.#.##.
...##.
.#..##
##.###

After 3 steps:
#...##
####.#
..##.#
......
##....
####.#

After 4 steps:
#.####
#....#
...#..
.##...
#.....
#.#..#

After 5 steps:
##.###
.##..#
.##...
.##...
#.#...
##...#
```

In [7]:
bgrid = BrokenGrid(example)
print(bgrid)
for i in range(5):
    print(f'\nStep {i + 1}:')
    bgrid.step()
    print(bgrid)
    print(f"Lamps on: {bgrid.countOn()}")


##.#.#
...##.
#....#
..#...
#.#..#
####.#

113241
333243
022331
241222
264431
243231

Step 1:
#.##.#
####.#
...##.
......
#...#.
#.####

254341
245562
234332
111332
132433
131332
Lamps on: 18

Step 2:
#..#.#
#....#
.#.##.
...##.
.#..##
##.###

121031
233352
213343
224454
324564
223243
Lamps on: 18

Step 3:
#...##
####.#
..##.#
......
##....
####.#

243332
244463
244341
233221
344221
343120
Lamps on: 18

Step 4:
#.####
#....#
...#..
.##...
#.....
#.#..#

131232
133452
233121
222210
253211
130110
Lamps on: 14

Step 5:
##.###
.##..#
.##...
.##...
#.#...
##...#

234232
455442
355311
354300
363211
232110
Lamps on: 17


In [8]:
bgrid = BrokenGrid(content)
for i in range(100):
    bgrid.step()
print(f"Lamps on: {bgrid.countOn()}")


Lamps on: 781
